## Applied Data Science Project

Hugo Bossi, Fabian Frieden, Domenic Largiadèr

### Choosing the appropriate alogrithm for prediction 

After we have now consolidated our data from a CSV File and from a NBA Website into a Dataframe, we would like to consider
the appropriate alogirthm for our prediction. We would like to predict what the probability is, that a NBA Team is qualifying for playoffs based on this historical data from 2011 to 2022. 

In [91]:
from sklearn.linear_model import LogisticRegression
#from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)


In [92]:
#select needed colums for model
# ORtg/A = Adjusted Offensive Rating = Points scored per 100 Possesion
# DRtg/A = Adjusted Defensive Rating = Points allowed per 100 Possesion

df_NBA = final_df[['ORtg/A','DRtg/A','playoffs']]
      
df_NBA   

,ORtg/A,DRtg/A,playoffs
Nr.,,,
1,117.42,115.84,0
2,116.33,114.14,0
3,108.18,115.56,0
4,108.83,114.59,0
5,105.81,111.12,0
...,...,...,...
356,107.03,110.54,0
357,101.68,105.11,0
358,107.18,105.70,1


In [93]:
#define target value and split test data

X = df_NBA.drop(['playoffs'],axis=1).values
y = df_NBA['playoffs'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state = 99)

In [94]:
log_reg = LogisticRegression()

log_reg.fit(X_train, y_train)

LogisticRegression()

In [95]:
print(log_reg.coef_)
print(log_reg.intercept_)

[[ 0.3626831  -0.68604809]]
[34.87533144]


In [96]:
mean_accuracy = log_reg.score(X_test, y_test)
print("The accuracy of your model is {0}%".format(np.round(mean_accuracy*100),2))

The accuracy of your model is 88.0%
